In [1]:
!pip install transformers
!pip install datasets

In [2]:
from datasets import load_dataset

nsmc = load_dataset('nsmc')
# naver sentiment movie corpus라는 이름의 데이터셋 다운로드.
# 영화 리뷰 댓글을 이용해 감정 분류하는 목적으로 제작된 데이터셋이다.
# 0 - 부정, 1 - 긍정

In [3]:
nsmc
# 로드한 데이터 확인
# train set - 150000개, test set - 50000개

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [4]:
train_data = nsmc['train'].shuffle(seed=42).select(range(2000))
test_data = nsmc['test'].shuffle(seed=42).select(range(2000))
# 전체 데이터를 학습시키려면 너무 오랜시간이 걸리기 때문에
# train data 2000개, test data 2000개를 무작위로 샘플링 할것.

In [5]:
train_data
# trian_data확인

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 2000
})

In [6]:
test_data
# test_data확인

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 2000
})

In [7]:
# model과 tokenizer로 ‘bert-base-multilingual-cased’을 사용
MODEL_NAME = 'bert-base-multilingual-cased'

# 문장이 어떤 감정에 해당하는 지 분류하는 것이기 때문에
# Sequence Classification을 위한 AutoClass인 AutoModelForSequenceClassification을 사용
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# tokenizer가 정상작동하는지 테스트
tokenizer.tokenize(train_data['document'][0])

['For',
 'Carl',
 '.',
 '칼',
 '세',
 '##이',
 '##건',
 '##으로',
 '시',
 '##작',
 '##해서',
 '칼',
 '세',
 '##이',
 '##건',
 '##으로',
 '끝',
 '##난',
 '##다',
 '.']

In [9]:
# data의 document를 모두 encoding한다.

train_encoding = tokenizer( # 훈련데이터의 문서들을 토큰화
    train_data['document'], # 훈련데이터에서 document열에 있는 텍스트 데이터를 사용.
    return_tensors='pt',    # 반환되는 결과가 PyTorch텐서로 저장되어야함.
    padding=True,           # 패딩을 수행.(패딩은 입력 시퀀스의 길이를 맞추기 위해 추가토큰을 삽입하는 과정.)
    truncation=True         # 필요한 경우 시퀀스를 잘라야함.
)

test_encoding = tokenizer(
    test_data['document'],
    return_tensors='pt',
    padding=True,
    truncation=True
)

In [10]:
# train과 test의 개수 확인

len(train_encoding['input_ids']), len(test_encoding['input_ids'])

(2000, 2000)

In [11]:
import torch
from torch.utils.data import Dataset


class NSMCDataset(Dataset):
    def __init__(self, encodings, labels): # 생성자 메서드 정의.
        self.encoding = encodings # 텍스트 데이터의 인코딩 정보를 포함하는 딕셔너리
        self.labels = labels # 각 텍스트 데이터에 대한 레이블

    def __getitem__(self, idx): # 데이터셋에서 특정 인덱스에 해당하는 항목을 반환하는 메서드 정의
        data = {key: val[idx] for key, val in self.encoding.items()} # 저장된 토큰화된 입력 데이터에서 idx번째의 데이터를 추출하여
                                                # 딕셔너리 'data'에 저장한다. 이 딕셔너리는 텍스트의 데이터 인코딩 정보를 포함한다

        data['labels'] = torch.tensor(self.labels[idx]).long() # idx번째 레이블을 추출하고 이를 PyTorch의 torch.tensor으로 변환하여
                                                # 정수데이터로 변환한다음 data딕셔너리에 labels키로 저장한다.

        return data

    def __len__(self): # 데이터셋의 전체데이터 포인트 수를 반환하는 메서드를 정의한다.
        return len(self.labels)

In [12]:
train_set = NSMCDataset(train_encoding, train_data['label'])
test_set = NSMCDataset(test_encoding, test_data['label'])

In [13]:
train_set[0]

{'input_ids': tensor([  101, 11399, 12225,   119,  9788,  9435, 10739, 71439, 11467,  9485,
         38709, 70146,  9788,  9435, 10739, 71439, 11467,  8977, 33305, 11903,
           119,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0,

In [14]:
test_set[0]

{'input_ids': tensor([  101, 14796, 27728, 10230,   106,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

# Trainer 클래스로 학습하기

   Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공합니다.

   TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision을 간단하게 설정해 학습, 평가를 모두 진행할 수 있습니다.

   


In [15]:
# Training에 필요한 argument 정의.
from transformers import TrainingArguments, Trainer

In [16]:
pip install accelerate


In [17]:
pip install accelerate -U


In [18]:
training_args = TrainingArguments(
    output_dir = './outputs', # model이 저장되는 directory
    logging_dir = './logs', # log가 저장되는 directory
    num_train_epochs = 10, # training epoch 수
    per_device_train_batch_size=32,  # train batch size
    per_device_eval_batch_size=32,   # eval batch size
    logging_steps = 50, # logging step, batch단위로 학습하기 때문에 epoch수를 곱한 전체 데이터 크기를 batch크기로 나누면 총 step 갯수를 알 수 있다.
    save_steps= 50, # 50 step마다 모델을 저장한다.
    save_total_limit=2 # 2개 모델만 저장한다.
)

In [20]:
# GPU학습을 위해 device를 cude로 설정합니다.
# TrainingArguments를 이용해 Trainer을 만듭니다.

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

Trainer 클래스는 별도의 metric을 제공해주지않기 때문에 별도의 함수를 통해 계산을 따로 해주어야합니다.

  -accuracy와 f1 score를 계산하기위한 compute_metrics 함수를 만듭니다.

1.해당 함수는 인자를 통해 EvalPrediction 객체를 넘겨 받습니다.
2.EvalPrediction은 predictions와 label_ids를 가집니다.
3.predictions: model의 예측값
4.label_ids: label 값
5.datasets에서 제공하는 load_metric()을 이용해 accuracy와 f1 score를 계산합니다.


In [21]:
from datasets import load_metric

def compute_metrics(pred): # compute_metrics라는 사용자 정의 함수를 정의. 이 함수는 모델의 예측 결과 pred를 인자로 받음.
    labels = pred.label_ids # pred에서 실제 레이블을 가져와 labels 변수에 저장.
    preds = pred.predictions.argmax(-1) # pred에서 모델의 예측중 가장 높은 확률을 가진 클래스의 인덱스를 가져와 preds에 저장.

    # load_metric함수를 사용하여 정확도를 계산할 객체를 초기화
    m1 = load_metric('accuracy')
    m2 = load_metric('f1')

    # predictions = 모델의 예측, references = 실제 레이블, 계산된 정확도는 accuracy키를 사용하여 접근
    acc = m1.compute(predictions=preds, references=labels)['accuracy'] # 메트릭 객체를 사용하여 정확도를 계산.
    f1 = m2.compute(predictions=preds, references=labels)['f1']

    return {'accuracy':acc, 'f1':f1}

In [22]:
# model 학습.

model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set, # 학습 세트
    eval_dataset=test_set, # 테스트 세트
    compute_metrics=compute_metrics # metric 계산 함수
)

In [23]:
trainer.train()

Step,Training Loss
50,0.662300
100,0.535800
150,0.426100
200,0.291500
250,0.209700
300,0.112700
350,0.123100
400,0.080600
450,0.079100
500,0.062300


TrainOutput(global_step=630, training_loss=0.2136222038950239, metrics={'train_runtime': 557.4549, 'train_samples_per_second': 35.877, 'train_steps_per_second': 1.13, 'total_flos': 1223055296400000.0, 'train_loss': 0.2136222038950239, 'epoch': 10.0})

- global_step=630: 글로벌 스텝은 가중치 업데이트가 발생하는 스텝의 수를 나타냄

- training_loss=0.2136222038950239: 훈련 중에 현재 스텝에서 계산된 훈련 손실. 손실은 모델이 예측과 실제 레이블 사이의 차이를 나타내며, 이 값이 작을수록 모델이 더 잘 수행했음을 나타냄.

- metrics: 훈련 중에 수집된 추가 메트릭(성능 지표) 정보가 포함된 딕셔너리.

- train_runtime: 전체 훈련 시간을

- train_samples_per_second: 훈련 속도를 나타내며, 초당 처리되는 훈련 데이터

- train_steps_per_second: 초당 처리되는 훈련 스텝 수를

- total_flos: 훈련 중에 수행된 총 부동 소수점 연산

- train_loss: 전체 훈련 과정에서 계산된 평균 훈련 손실

- epoch: 에폭은 훈련 데이터셋을 한 번 완전히 통과하는 횟수

In [24]:
# model 평가.
trainer.evaluate()

<ipython-input-21-7d79e22d4ac8>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  m1 = load_metric('accuracy')


{'eval_loss': 1.2342458963394165,
 'eval_accuracy': 0.769,
 'eval_f1': 0.7737512242899118,
 'eval_runtime': 17.5651,
 'eval_samples_per_second': 113.862,
 'eval_steps_per_second': 3.587,
 'epoch': 10.0}

- eval_loss: 평가 중 계산된 평가 손실. 이 값은 모델이 테스트 데이터에서 얼마나 좋은 예측을 하는지를 나타내는 지표이며, 낮을수록 모델의 성능이 더 좋음.

- eval_accuracy: 평가 중 계산된 정확도 (accuracy). 이 값은 모델의 예측이 실제 레이블과 일치하는 비율임. 1에 가까울수록 모델이 더 정확하게 예측하는 것.

- eval_f1: 평가 중에 계산된 F1 점수. F1 점수는 정밀도(Precision)와 재현율(Recall)을 결합한 지표로, 모델의 정확성과 불확실성을 고려하여 예측의 품질을 측정.

- eval_runtime: 평가 시간.

- eval_samples_per_second: 초당 처리된 평가 데이터 포인트 수.

- eval_steps_per_second: 초당 평가 스텝 수.

- epoch: 모델이 전체 테스트 데이터셋을 한 번 완전히 통과한 횟수.

# PyTorch Native 방식으로 학습하기

In [25]:
#TrainingArguments와 Trainer를 사용하지않고 학습하는 방법.
#먼저 DataLoader를 만듦.
# batch size = 32.
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [26]:
# 다음으로는 train 함수를 구성할 것입니다.
# train 함수를 구성하려면 model의 output이 어떻게 구성되었는지 확인해야합니다.
# dummy 데이터를 이용해 model의 output이 어떻게 나오는 지 확인해봅니다.

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
dummy = tokenizer(train_data['document'][0], return_tensors='pt')

model(**dummy)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3471, -0.3717]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

- model의 output에서 주목할 것은 loss와 logits입니다. logits는 model의 예측값을 가리킵니다. loss는 말그대로 loss 값을 가리킵니다.

- label이 없기 때문에 loss가 계산되지 않았습니다.
이 2가지 정보를 이용해 train 함수를 구성하겠습니다.

In [37]:
from tqdm.notebook import tqdm
from datasets import load_metric

def train(epoch, model, dataloader, optimizer, device):
    # epoch: 훈련 에폭 수를 나타내는 정수.
    # model: 훈련할 모델.
    # dataloader: 훈련 데이터를 불러오는 데이터로더.
    # optimizer: 모델의 가중치를 업데이트하는 데 사용되는 옵티마이저.
    # device: 모델을 훈련하는 데 사용되는 장치 (예: "cuda" 또는 "cpu").

    model.to(device) #  모델을 지정된 정치로 이동.

    m1 = load_metric('accuracy') # load_metric함수를 사용하여 정확도와 F1메트릭 객체를 초기화시킴.
    m2 = load_metric('f1')       # 이 메트릭 객체는 모델의 훈련 및 평가중 사용됨.

    for e in range(1, epoch+1): # 주어진에포크 수에 대한 반복문 시작.
        total_loss = 0.         # epoch내에서 손실을 누적하고
        preds = []              # 모델의 예측 및 실제 레이블을 추적하기위한 변수 초기화.
        labels = []
        progress_bar = tqdm(dataloader, desc=f'TRAIN - EPOCH {e} |')
        # 데이터 로더의 진행 상황을 모니터링할 진행률 바 생성.

        for data in progress_bar:
            data = {k:v.to(device) for k, v in data.items()} # 각 미니배치(data)내의 모든 데이터와 텐서를 지정된 장치로 이동.
            output = model(**data) # model에 data를 전달하여 예측결과를 얻습니다.(**data = 딕셔너이형식의  데이터를 함수의키워드인수로 전달하는 방법)
            current_loss = output.loss # 미니배치의 손실을 나타냄.

            total_loss += current_loss # epoch내에서 전체 손실을 추적하는데 사용.
            preds += list(output.logits.argmax(-1)) # logits는 모델의 클래스별 점수. argmax(-1)은 각 예측에서 가장 높은점수를 가진 클래스의 인덱스
            labels += list(data['labels'].detach().cpu().numpy())
            # 실제 레이블 추출, labels 리스트에 추가, #dtach()는 텐서에서 계산그래프 분리.cpu() = cpu이동, .numpy() = NumPy배열로 변환

            optimizer.zero_grad() # 옵티마이저의 그래디언트 초기화.(이전의 결과가 현재의 결과에 영향을 미치지 않기위함.)
            current_loss.backward() # 현재 미니배치의 손실에 대한 역전파를 수행. 이를 사용하여 모델의 가중치 업데이트.
            optimizer.step() # 옵티마이저를 사용하여 모델의 가중치 업데이트. 역전파로 계산된 그래디언트를 사용하여 파라미터 조정.

            progress_bar.set_description(f'TRAIN - EPOCH {e} | current-loss: {current_loss:.4f}')
            # 진행바 표시.


        acc = m1.compute(predictions=preds, references=labels)['accuracy'] # 모델의 예측과 실제 레이블을 비교하여 정확도 계산.
        f1 = m2.compute(predictions=preds, references=labels)['f1'] # 모델의 예측과 실제 레이블을 비교하여 F1점수를 계산.
                                  # F1점수 : 모델의 정밀도와 재현율을 결합한 성능지표. 정확성,불확실성을 고려하여 계산됨
        avg = total_loss / len(dataloader) # 전체 epoch동안 누적손실을 미니배치 수로 나눠서 평균손실을 계산.
                                           # 모델의 훈련중 손실이 어떻게 변화하는지를 나타낸다.
        print('='*64)
        print(f"TRAIN - EPOCH {e} | LOSS: {avg:.4f} ACC: {acc:.4f} F1: {f1:.4f}") # 현제 epoch에서 훈련결과를 출력.
        print('='*64)

In [39]:
# 위코드와 유사함. (epoch, backward과정이 생략됨.)

def evaluate(model, dataloader, device):
    model.to(device)

    m1 = load_metric('accuracy')
    m2 = load_metric('f1')

    total_loss = 0.
    preds = []
    labels = []
    progress_bar = tqdm(dataloader, desc=f'EVAL |')
    for data in progress_bar:
        data = {k:v.to(device) for k, v in data.items()}

        with torch.no_grad():
            output = model(**data)

        current_loss = output.loss

        total_loss += current_loss
        preds += list(output.logits.argmax(-1))
        labels += list(data['labels'].detach().cpu().numpy())

        progress_bar.set_description(f'EVAL | current-loss: {current_loss:.4f}')

    acc = m1.compute(predictions=preds, references=labels)['accuracy']
    f1 = m2.compute(predictions=preds, references=labels)['f1']
    avg = total_loss / len(dataloader)

    print('='*64)
    print(f"EVAL | LOSS: {avg:.4f} ACC: {acc:.4f} F1: {f1:.4f}")
    print('='*64)

In [41]:
# pytorch를 이용해 학습진행


from torch.optim import AdamW

# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

In [42]:
train(10, model, train_loader, optimizer, device)

TRAIN - EPOCH 1 |:   0%|          | 0/63 [00:00<?, ?it/s]

TRAIN - EPOCH 1 | LOSS: 0.7010 ACC: 0.4980 F1: 0.4743


TRAIN - EPOCH 2 |:   0%|          | 0/63 [00:00<?, ?it/s]

TRAIN - EPOCH 2 | LOSS: 0.6969 ACC: 0.4860 F1: 0.4776


TRAIN - EPOCH 3 |:   0%|          | 0/63 [00:00<?, ?it/s]

...

TRAIN - EPOCH 8 | LOSS: 0.6951 ACC: 0.4900 F1: 0.4138


TRAIN - EPOCH 9 |:   0%|          | 0/63 [00:00<?, ?it/s]

TRAIN - EPOCH 9 | LOSS: 0.6950 ACC: 0.5000 F1: 0.4033


TRAIN - EPOCH 10 |:   0%|          | 0/63 [00:00<?, ?it/s]

TRAIN - EPOCH 10 | LOSS: 0.6949 ACC: 0.4990 F1: 0.3905


In [43]:
# Trainer을 활용해 학습한것과 비슷한 결과를 얻은것을 확인할 수 있다.
evaluate(model, test_loader, device)

EVAL |:   0%|          | 0/63 [00:00<?, ?it/s]

EVAL | LOSS: 0.6958 ACC: 0.4835 F1: 0.6518
